In [ ]:
import pandas as pd
import itertools

In [ ]:
df = pd.read_csv('../data/netflix_titles_raw.csv') 
df.date_added = pd.to_datetime(df.date_added)
df.listed_in = df.listed_in.str.split(',')
df.show_id = df.show_id.str[1:]
df

# Get all unique person entities

In [ ]:
cast_members = list(df.cast.str.split(',').dropna())
directors = list(df.director.str.split(',').dropna())
persons = pd.DataFrame(set(person.strip() for person in itertools.chain(*cast_members + directors)),
                      columns=['person']
                     )
persons['person_id'] = persons.index + 1

persons

# Intersection table for cast and show

In [ ]:
def show_cast_intersection_record_creation(record):
    return [(record.show_id, person.strip()) for person in record.cast.split(',')]

show_cast_intersection_records = []
for record in df[df.cast.notna()].itertuples():
    if record.cast:
        show_cast_intersection_records.extend(show_cast_intersection_record_creation(record))
    
show_cast_intersection_df = pd.DataFrame(
    show_cast_intersection_records, 
    columns=['show_id', 'person']
)
show_cast_intersection_df = show_cast_intersection_df.merge(persons)

del show_cast_intersection_df['person']
show_cast_intersection_df


# Create a director_id column in the raw dataset

In [ ]:
df = df.merge(persons, left_on='director', right_on='person', how='left')
df['director_id'] = df.person_id.astype('Int64')
df

# Export person and show/cast intersection tables

In [ ]:
persons.to_csv('../data/persons.csv', index=False)
show_cast_intersection_df.to_csv('../data/show_cast_intersection.csv', index=False)

# Create a table of unique categories

In [ ]:
categories = pd.DataFrame(set(category.strip() for category in itertools.chain(*list(df.listed_in))),
                          columns=['category']
                         )
categories['category_id'] = categories.index + 1

categories

# Intersection table between category and show

In [ ]:
def show_category_intersection_record_creation(record):
    return [(record.show_id, category.strip()) for category in record.listed_in]

show_category_intersection_records = []
for record in df.itertuples():
    show_category_intersection_records.extend(show_category_intersection_record_creation(record))
    
show_category_intersection_df = pd.DataFrame(
    show_category_intersection_records, 
    columns=['show_id', 'category']
)

show_category_intersection_df = show_category_intersection_df.merge(categories)
del show_category_intersection_df['category']
show_category_intersection_df

# Export categories and show/categories intersection tables

In [ ]:
show_category_intersection_df.to_csv('../data/show_category_intersection.csv', index=False)
categories.to_csv('../data/categories.csv', index=False)

# Finally, create the show table

In [ ]:
show_df = df[['show_id','type', 'title', 'director_id', 'country', 'date_added', 'release_year', 'rating', 'duration', 'description']]
show_df

# Export it

In [ ]:
show_df.to_csv('../data/shows.csv', index=False)